# Amazon SageMaker XGBoostによるターゲティングダイレクトマーケティング
_**Gradient Boosted Treesによる教師付き学習：バランスの取れていないクラスでの二値予測問題**_

---

---

## 目次

1. [背景](#Background)
1. [準備](#Preparation)
1. [データ](#Data)
    1. [探索](#Exploration)
    1. [変換](#Transformation)
1. [トレーニング](#Training)
1. [ホスティング](#Hosting)
1. [評価](#Evaluation)
1. [自動モデルチューニング（オプション）](#Automatic-model-Tuning-(optional))
1. [拡張](#Extensions)

---

## Background

ダイレクトマーケティングは、郵便物や電子メール、電話などを使って顧客を獲得するための一般的な手法です。リソースやお客様の関心は限られているため、特定のオファーに興味を持ってくれそうな見込み客のサブセットのみをターゲットにすることが目標となります。人口統計、過去のインタラクション、環境要因など、すぐに入手できる情報に基づいて、それらの見込み客を予測することは、一般的な機械学習の問題です。

このノートでは、1回以上の電話連絡の後、顧客が銀行の定期預金に登録するかどうかを予測する問題の例を紹介します。手順は以下の通りです。

* Amazon SageMakerノートブックの準備
* インターネットからAmazon SageMakerへのデータのダウンロード
* Amazon SageMakerのアルゴリズムにデータを供給できるように、データを調査して変換する
* Gradient Boostingアルゴリズムを使用したモデルの推定
* モデルの有効性を評価する
* 継続的な予測を行うためのモデルの設定

---

## Preparation

_このノートブックは、ml.m4.xlargeノートブックのインスタンスで作成され、テストされました。_

まずは以下を指定してみましょう。

- トレーニングデータとモデルデータに使用するS3バケットとプレフィックスを指定します。これは、ノートブックインスタンス、トレーニング、およびホスティングと同じリージョン内にある必要があります。
- トレーニングとホスティングにデータへのアクセスを与えるために使用するIAMロールarn。これらの作成方法については、ドキュメントを参照してください。 ノートブックインスタンス、トレーニング、および/またはホスティングに複数のロールが必要な場合は、botoの正規表現を適切な完全なIAMロールarnの文字列に置き換えてください。

In [ ]:
conda update pandas

In [ ]:
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

それでは、解析に使用するPythonライブラリを導入しましょう。

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [ ]:
pd.__version__

pandasのバージョンが1.2.4以降であることを確認してください。そうでない場合は、先に進む前にカーネルを再起動してください。

---

## Data

まずは、サンプルデータのs3バケットから[direct marketing dataset](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip)をダウンロードしてみましょう。


\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014


In [ ]:
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

では、これをPandasのデータフレームに読み込んで見てみましょう。

In [ ]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

データについて説明しましょう。 大まかには以下のようになります。

* 4万以上の顧客レコードがあり、各顧客に20の特徴量があります。
* 特徴量には、数値とカテゴリーが混在しています。
* データは、少なくとも `time` と `contact` でソートされているように見えますが、もっとあるかもしれません。

_**各特徴の詳細： **_

*人口統計:*
* `age`: 顧客の年齢(数値)
* `job`: 仕事の種類 (カテゴリカル: 'admin.', 'service', ...)
* `marital`: 配偶者の有無 (カテゴリカル: 'married', 'single', ...)
* `education`: 教育水準 (カテゴリカル: 'basic.4y', 'high.school', ...)

*過去の顧客イベント:*
* `default`: クレジットがデフォルトになったことがあるか？(カテゴリカル: 'no', 'unknown', ...)
* `housing`: 住宅ローンを利用しているか？(カテゴリカル: 'no', 'yes', ...)
* `loan`: 個人ローンを利用しているか？(カテゴリカル: 'no', 'yes', ...)

*過去のダイレクトマーケティングの連絡:*
* `contact`: 連絡の通信タイプ (カテゴリカル: 'cellular', 'telephone', ...)
* `month`: 最後に連絡を取った月 (カテゴリカル: 'may', 'nov', ...)
* `day_of_week`: 最後に連絡を取った曜日 (カテゴリカル: 'mon', 'fri', ...)
* `duration`: 直近の接触時間を秒単位で表したもの (数値)。重要: duration = 0 の場合、`y` = 'no' となる。
 
*キャンペーン情報:*
* `campaign`: このキャンペーンで、このクライアントに対して行われたコンタクトの数 (数値、最後のコンタクトを含む)
* `pdays`: クライアントが前回のキャンペーンで最後に連絡を取ってからの経過日数 (数値)
* `previous`: このキャンペーンの前に、このクライアントに対して行われたコンタクトの数 (数値)
* `poutcome`: 前回のマーケティングキャンペーンの結果 (カテゴリカル： 'nonexistent', 'success', ...)

*外部環境要因:*
* `emp.var.rate`: 雇用変動率 - 四半期ごとの指標 (数値)
* `cons.price.idx`: 消費者物価指数 - 月次指標 (数値)
* `cons.conf.idx`: 消費者信頼感指数 - 月次指標 (数値)
* `euribor3m`: Euribor3ヶ月レート - 日次指標 (数値)
* `nr.employed`: 従業員数 - 四半期ごとの指標(数値)

*目的変数:*
* `y`: 顧客が定期預金を申し込んだか？ (バイナリー: 'yes', 'no')

### Exploration

早速、データを調べてみましょう。 まず、特徴量がどのように分布しているかを理解しましょう。

In [ ]:
# Frequency tables for each categorical feature
for column in data.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=data[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(data.describe())
%matplotlib inline
hist = data.hist(bins=30, sharey=True, figsize=(10, 10))

次のことに注目してください。

* ターゲット変数 `y` の値のほぼ90%が「no」で、ほとんどの顧客が定期預金に加入していないことがわかります。
* 予測特徴量の多くは「unknown」という値をとります。あるものは他のものよりも一般的です。何が「unknown」の値の原因なのか（これらの顧客は何らかの形で少数派なのか）、そしてそれをどのように扱うべきなのかを注意深く考える必要があります。
  * 「unknown」が独自のカテゴリーとして含まれているとしても、実際には、それらの観測結果がその特長量の他のカテゴリーのいずれかに該当する可能性が高いことを考えると、それは何を意味するのでしょうか？
* 予測特徴量の多くは、その中に観測値が非常に少ないカテゴリーを持っています。小さなカテゴリーがターゲットとする結果の予測率が高いと分かった場合、それについて一般化するのに十分な証拠があるのでしょうか？
* 連絡のタイミングは特に偏っています。5月はほぼ3分の1、12月は1％未満です。これは、来年の12月に目標とする変数を予測する上で、どのような意味を持つのでしょうか？
* 数値特徴量に欠損値はありません。あるいは、欠損値はすでに補完されています。
  * `pdays` は、ほぼすべての顧客で1000に近い値をとります。おそらく、過去にコンタクトがなかったことを示すプレースホルダー値と思われます。
* いくつかの数値特徴量は、非常に長いテールを持っています。これらの極端に大きな値を持つ少数の観測値を別の方法で処理する必要がありますか？
* いくつかの数値特徴（特にマクロ経済的なもの）は、異なるバケットで発生します。 これらはカテゴリーとして扱うべきでしょうか？

次に、特徴量が予測しようとしているターゲットとどのように関連しているかを見てみましょう。

In [ ]:
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':
        display(pd.crosstab(index=data[column], columns=data['y'], normalize='columns'))

for column in data.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = data[[column, 'y']].hist(by='y', bins=30)
    plt.show()

以下の点にご注意ください。

* 「blue-collar」、「married」、デフォルト（債務不履行）が「unknown」、「telephone」で連絡を受けたことがある、「may」に連絡を受けたことがある、などの顧客は、「yes」が「no」よりも大幅に少ない。
* 数値変数の分布は、「yes」と「no」の購読グループで異なっているが、その関係は単純明快なものではないかもしれない。

次に、特徴量がお互いにどのように関連しているかを見てみましょう。

In [ ]:
display(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

以下に注目してください。
* 特徴量間の関係はモノによって大きく異なる。負の相関が強いものもあれば、正の相関が強いものもある。
* 特徴量間の関係は非線形であり、多くの場合、離散的である。

### Transformation

データのクリーンアップは、ほぼすべての機械学習プロジェクトに含まれます。この作業は、間違った方法で行われた場合には最大のリスクとなり、プロセスの中でも最も主観的な側面の1つとなります。一般的な手法としては以下のようなものがあります。

* 欠損値の処理：機械学習アルゴリズムの中には、欠損値を扱うことができるものもありますが、ほとんどの場合はそうではありません。オプションは以下の通りです。
 * 欠損値のある観測値を削除する：これは、不完全な情報を持つ観測値がごく一部であればうまくいく。
 * 欠損値を持つ特徴量を削除する：これは、多数の欠損値を持つ特徴量のカラムが少数の場合にうまくいく。
 * 欠損値の代入：これは[書籍](https://www.amazon.com/Flexible-Imputation-Missing-Interdisciplinary-Statistics/dp/1439868247)全体で語られるようなトピックですが、一般的な選択肢は、欠損値をそのカラムの非欠損値の最頻値または平均値で置き換えることです。
* カテゴリーを数値に変換する：最も一般的な方法は、ホットエンコーディングと呼ばれるもので、各特徴量について、そのカラムの異なる値をそれぞれの特徴量にマッピングし、カテゴリー特徴量がその値と等しい場合は1、そうでない場合は0の値を取るものです。
* 奇妙な分布のデータ：Gradient Boosted Treesのような非線形モデルでは、この問題は非常に限定的ですが、回帰のようなパラメトリックモデルでは、非常に歪んだデータを入力すると、非常に不正確な推定値を生成する可能性があります。場合によっては、特徴量の自然対数を取るだけで、より普通に分布したデータが得られることもあります。また、値を離散的な範囲にバケット化することが有効な場合もあります。これらのバケットはカテゴリー変数として扱われ、1つのホットエンコードされたモデルに含まれます。
* より複雑なデータタイプを扱う：画像、テキスト、粒度の異なるデータを扱う場合は、他のノートブックテンプレートを使用します。

幸いなことに、これらの側面のいくつかはすでに処理されており、今回紹介するアルゴリズムは、まばらなデータや奇妙な分布のデータをうまく扱う傾向があります。そこで、前処理をシンプルにしておきましょう。

In [ ]:
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed
model_data = pd.get_dummies(data)                                                                  # Convert categorical variables to sets of indicators

モデルを構築する前のもう一つの質問は、ある特徴量が最終的なユースケースで価値を付加するかどうかです。例えば、最高の予測を提供することが目的であれば、予測の瞬間にそのデータにアクセスできるでしょうか。 雨が降っていることを知っていれば、傘の売れ行きを予測することができますが、傘の在庫を計画するために十分に先の天気を予測することは、天気を知らずに傘の売れ行きを予測するのと同じくらい難しいでしょう。そのため、これをモデルに含めると、精度が高いと勘違いしてしまう可能性があります。

この論理に従って、将来の予測の入力として使用するには高い精度で予測する必要があるため、データから経済的特徴と `duration` を削除してみましょう。

仮に前四半期の経済指標の値を使用したとしても、この値は次の四半期の早い時期に連絡を受けた見込み客にとっては、遅い時期に連絡を受けた見込み客ほど関連性がないと思われます。

In [ ]:
model_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)

新しいデータに対してターゲット値を予測することを主な目的としたモデルを構築する際には、オーバーフィッティングを理解することが重要です。教師あり学習モデルは、ターゲット値の予測と、与えられたデータにおける実際の値との誤差を最小化するように設計されています。ここが鍵となります。機械学習モデルは、より高い精度を追求するあまり、与えられたデータの中の些細な特異性を拾うように偏ってしまうことがよくあります。このような特異性は次のデータでは再現されないため、訓練段階でのより正確な予測を犠牲にして、実際には予測の精度が低くなってしまうのです。

これを防ぐための最も一般的な方法は、「モデルは、学習したデータへの適合性だけでなく、「新しい」データへの適合性も含めて判断されるべきである」というコンセプトでモデルを構築することです。ホールドアウトバリデーション、クロスバリデーション、リーブワンアウトバリデーションなど、いくつかの異なる方法でこの概念を表現することができます。ここでは、データをランダムに3つのグループに分けることにします。70%のデータでモデルを訓練し、20%のデータで評価して「新しい」データでの精度を推定し、10%は最終的なテストデータセットとして保留しておきます。

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

Amazon SageMakerのXGBoostコンテナは、libSVMまたはCSVデータ形式のデータを想定しています。この例では、CSVを用います。最初のカラムはターゲット変数でなければならず、CSVにはヘッダーが含まれてはならないことに注意してください。また、繰り返しになりますが、トレーニング、バリデーション、テストの分割の前ではなく、分割後に行うのが最も簡単であることに注意してください。これにより、ランダムな並び替えによるミスアライメントの問題を避けることができます。

In [ ]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

ここで、Amazon SageMakerのマネージドトレーニングがファイルをピックアップできるように、ファイルをS3にコピーします。

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## ラボ1の終わり


---

## Training

今、私たちは、ほとんどの特徴量が歪んだ分布を持ち、いくつかの特徴量は互いに高い相関を持ち、いくつかの特徴量はターゲット変数と非線形の関係を持っているように見えることを知っています。また、将来の見込み客を対象とする場合、予測精度が高いことよりも、なぜその見込み客を対象としたのかを説明できることが重要です。このような点を考慮すると、gradient boosted treesは良いアルゴリズムの候補と言えます。

このアルゴリズムを理解するにはいくつかの複雑な点がありますが、高いレベルでは、gradient boosted treesは、多くの単純なモデルからの予測を組み合わせることで機能します。各モデルは、以前のモデルの弱点を解決しようとします。このようにして、単純なモデルの集まりが、大規模で複雑なモデルよりも優れた結果を出すことができるのです。Amazon SageMakerの他のノートブックでは、gradient boosted treesについてさらに詳しく説明されており、類似のアルゴリズムとの違いについても説明されています。

`xgboost`は、非常に人気のある、gradient boosted treesのオープンソースパッケージです。計算能力が高く、機能も充実しており、多くの機械学習のコンテストで使用されています。まずは、Amazon SageMakerのマネージド分散学習フレームワークを使って訓練した、シンプルな`xgboost`モデルから始めましょう。

最初に、Amazon SageMakerのXGBoostの実装用にECRコンテナの場所を指定する必要があります。

In [ ]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

また、今回はCSVファイル形式で学習するので、S3内のファイルへのポインタとして訓練関数が使えるように `s3_input` を作成し、コンテンツタイプをCSVと指定します。

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

まず、estimatorの訓練パラメータを指定する必要があります。これは、
1. `xgboost`アルゴリズムのコンテナ
1. 使用するIAMロール
1. トレーニングインスタンスのタイプと数
1. 出力データのS3ロケーション
1. アルゴリズムのハイパーパラメータ

そして、以下を指定する `.fit()` 関数を含みます。
1. 出力データのS3ロケーション。 この例では、トレーニングセットとバリデーションセットの両方が渡されます。


In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## Hosting

さて、データに対して `xgboost` アルゴリズムをトレーニングしたので、リアルタイムエンドポイントを作成してモデルをデプロイしてみましょう。

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---

## Evaluation

機械学習モデルの性能を比較する方法はいろいろありますが、まずは単純に実績値と予測値を比較してみましょう。今回のケースでは、顧客が定期預金に加入しているか (`1`)、していないか (`0`) を予測するだけなので、単純な混同行列を生成します。

まず、エンドポイントへのデータの受け渡し方法を指定する必要があります。現在、データはNumPyの配列としてノートブックインスタンスのメモリに格納されています。HTTP POSTリクエストでデータを送信するには、データをCSV文字列としてシリアライズし、その結果のCSVをデコードします。

*注：CSV形式で推論する場合、SageMaker XGBoostではデータにターゲット変数が含まれていてはいけません。*

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

では、以下を行う簡単な関数を使ってみましょう。
1. テストデータセットのループ
1. 行のミニバッチに分割 
1. これらのミニバッチをCSV文字列のペイロードに変換する（最初にデータセットからターゲット変数を削除していることに注意してください）
1. XGBoostエンドポイントを起動してミニバッチの予測値を取得
1. 予測値を収集し、モデルが提供するCSV出力からNumPy配列に変換する

In [ ]:
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(), xgb_predictor)

ここで、混乱行列をチェックして、予測と実際の結果を確認します。

In [ ]:
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

つまり、約4000人の潜在顧客のうち、136人が購読すると予測し、そのうち94人が実際に購読したのです。また、予測していなかった389人の加入者がいました。 これは望ましいことではありませんが、モデルを調整して改善することができます（すべきです）。最も重要なことは、最小限の努力で、私たちのモデルが[こちら](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf)で発表されたものと同様の精度を出したことです。

_アルゴリズムのサブサンプルにはランダムな要素があるため、結果は上の文章とは若干異なる可能性があることに注意してください。_

## Automatic model Tuning (optional)

Amazon SageMakerの自動モデルチューニングは、ハイパーパラメータチューニングとしても知られており、指定したアルゴリズムとハイパーパラメータの範囲を使用してデータセット上でたくさんのトレーニングジョブを実行することにより、モデルのベストバージョンを見つけます。そして、選択した指標で測定した結果、最高のパフォーマンスを発揮するモデルになるハイパーパラメータの値を選択します。

例えば、このマーケティングデータセットの2値分類問題を解くとします。あなたの目的は、XGBoost アルゴリズムのモデルを訓練することで、アルゴリズムの曲線下面積（auc）の指標を最大化することです。eta、alpha、min_child_weight、max_depth の各ハイパーパラメータのどの値を使用すれば、最適なモデルを訓練できるかわかりません。これらのハイパーパラメータの最適な値を見つけるために、Amazon SageMakerのハイパーパラメータチューニングが検索する値の範囲を指定して、選択した目標指標で測定された最高のパフォーマンスを示す値の組み合わせを見つけることができます。ハイパーパラメータチューニングは、指定した範囲のハイパーパラメータ値を使用するトレーニングジョブを起動し、最も高いAUCを持つトレーニングジョブを返します。

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [ ]:
objective_metric_name = 'validation:auc'

In [ ]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)


In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# return the best training job name
tuner.best_training_job()

In [ ]:
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# Predict
predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [ ]:
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

---

## Extensions

この例では、比較的小さなデータセットを分析していますが、分散マネージドトレーニングやリアルタイムのモデルホスティングなど、Amazon SageMakerの機能を利用しており、より大きな問題にも容易に適用できます。予測精度をさらに向上させるためには、偽陽性と偽陰性の組み合わせを変えるために、予測のしきい値を微調整したり、ハイパーパラメータチューニングのような技術を検討したりすることができます。実際のシナリオでは、手作業で特徴量を作成する時間を増やしたり、最初のデータセットでは得られなかった顧客情報を含む追加のデータセットを探したりすることになるでしょう。

### (Optional) Clean-up

このノートブックが終わったら、以下のセルを実行してください。これにより、作成したエンドポイントが削除され、迷子のインスタンスがオンになったままになっていることによる課金を防ぐことができます。

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

In [ ]:
tuner_predictor.delete_endpoint(delete_endpoint_config=True)